## Problem: Labyrinth Navigation with Teleports

Date: 9/19/2025

**Setup:**
You're exploring a rectangular labyrinth (n × m matrix) containing obstacles and teleports. Start at (0, 0), goal is to reach (n-1, m-1).

**Movement Rules:**
1. **Always try RIGHT first**: From (row, col), try moving to (row, col+1)
2. **If RIGHT blocked/invalid, try DOWN**: Move to (row+1, col)
3. **If both blocked, STOP**

**Special Mechanics:**

**Obstacles:**
- Cannot traverse through obstacles
- If obstacle blocks your path, try the alternative direction
- If both directions blocked, stop immediately

**Teleports:**
- Represented as pairs: (start_cell, end_cell)
- If you step on a `start` cell, you **immediately teleport** to the `end` cell
- **One-way only**: Cannot teleport from `end` to `start`
- Each cell has at most one teleport starting from it
- The `end` cell of a teleport cannot be the `start` of another teleport
- Both `start` and `end` cells do NOT contain obstacles

**Free Cells:**
- Any cell without obstacle or teleport can be walked through normally

---

**Task:**
Return the **total number of cells traveled** (including starting cell (0,0) and both start/end cells of teleports).

**Return:**
- Total cell count if you reach (n-1, m-1)
- `-1` if blocked by obstacle or out of bounds
- `-2` if infinite teleport loop detected

**Complexity:** O(n · m · (obstacles.length + teleports.length)) is acceptable

---

**Example:**
- `n = 3, m = 3`
- `obstacles = [[2, 1]]`
- `teleports = [[0, 1, 2, 0]]` (from (0,1) to (2,0))
- Output: `-1`

The grid looks like:
```
[0,0] [0,1] [0,2]
[1,0] [1,1] [1,2]
[2,0]  X    [2,2]
```

---

## Initial Analysis

This is a **simulation problem** - you're literally following the stated algorithm step by step. The tricky parts are:

1. **Teleport detection** - need fast lookup for "is this cell a teleport start?"
2. **Cycle detection** - infinite teleport loops
3. **Following the priority** - right first, then down, then stop

This screams for a **state-based simulation** where we track position and visited states.

Want to talk through the approach, or take a crack at it first?

In [ ]:
def solution(n, m, obstacles, teleports):
    obs = {tuple(o) for o in obstacles}
    tp = {(t[0], t[1]): (t[2], t[3]) for t in teleports}
    
    r = c = 0
    steps = 1
    max_steps = n * m * 100  # Large enough to detect infinite loops
    
    while steps < max_steps:
        if r == n - 1 and c == m - 1:
            return steps
        
        moved = False
        
        # Try right
        if c + 1 < m and (r, c + 1) not in obs:
            c += 1
            steps += 1
            # Chain all teleports
            while (r, c) in tp:
                r, c = tp[(r, c)]
                steps += 1
            moved = True
        
        # Try down
        elif r + 1 < n and (r + 1, c) not in obs:
            r += 1
            steps += 1
            # Chain all teleports
            while (r, c) in tp:
                r, c = tp[(r, c)]
                steps += 1
            moved = True
        
        if not moved:
            return -1
    
    return -2  # Infinite loop detected